## **##### 1.Importing necessary libraries**

In [1]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import base64
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import gc

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import optuna
import xgboost as xgb
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, 
                              ExtraTreesClassifier, VotingClassifier)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn import datasets, linear_model
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

pd.set_option('display.max_columns', None)   

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import eli5
from eli5.sklearn import PermutationImportance

from sklearn.model_selection import GridSearchCV

## **##### 2.Loading datasets**

In [2]:
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

In [3]:
train_test = pd.concat([train, test], ignore_index=True)

In [4]:
Expenses_columns = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

In [5]:
train_test['Expenses'] = train_test.loc[:,Expenses_columns].sum(axis=1)

In [6]:
train_test.loc[:,['CryoSleep']]=train_test.apply(lambda x: True if x.Expenses == 0 and pd.isna(x.CryoSleep) else x,axis =1)

In [7]:
train_test.loc[:,['Group']] = train_test.PassengerId.apply(lambda x: x[0:4] )

In [8]:
train_test[['Deck', 'Number', 'Side']] = train_test['Cabin'].str.split('/', expand=True)

In [9]:
train_test.loc[:,['FirstName']] = train_test.Name.str.split(" ",expand=True).iloc[:,0]
train_test.loc[:,['SecondName']] = train_test.Name.str.split(" ",expand=True).iloc[:,1]
train_test['Name_key']=train_test['SecondName']+train_test['Group']

In [10]:
train_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Expenses,Group,Deck,Number,Side,FirstName,SecondName,Name_key
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0.0,0001,B,0,P,Maham,Ofracculy,Ofracculy0001
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,736.0,0002,F,0,S,Juanna,Vines,Vines0002
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,10383.0,0003,A,0,S,Altark,Susent,Susent0003
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,5176.0,0003,A,0,S,Solam,Susent,Susent0003
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1091.0,0004,F,1,S,Willy,Santantines,Santantines0004


In [11]:
num_cols = ['ShoppingMall','FoodCourt','RoomService','Spa','VRDeck','Expenses','Age']
cat_cols = ['CryoSleep','Deck','Side','VIP','HomePlanet','Destination', ]
transported=['Transported']

In [12]:
train_test = train_test[num_cols+cat_cols+transported].copy()

In [13]:
num_imp = SimpleImputer(strategy='mean')
cat_imp = SimpleImputer(strategy='most_frequent')

In [14]:
train_test[num_cols] = pd.DataFrame(num_imp.fit_transform(train_test[num_cols]),columns=num_cols)
train_test[cat_cols] = pd.DataFrame(cat_imp.fit_transform(train_test[cat_cols]),columns=cat_cols)

In [15]:
ohe = OneHotEncoder (handle_unknown='ignore',sparse = False)

In [16]:
temp_train = pd.DataFrame(ohe.fit_transform(train_test[cat_cols]),columns=ohe.get_feature_names_out())

In [17]:
train_test = train_test.drop(cat_cols,axis=1)

In [18]:
train_test = pd.concat([train_test,temp_train],axis=1)

In [19]:
train_test

,ShoppingMall,FoodCourt,RoomService,Spa,VRDeck,Expenses,Age,Transported,CryoSleep_False,CryoSleep_True,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S,VIP_False,VIP_True,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0.0,0.0,0.0,0.0,0.0,0.0,39.000000,False,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,25.0,9.0,109.0,549.0,44.0,736.0,24.000000,True,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,3576.0,43.0,6715.0,49.0,10383.0,58.000000,False,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,371.0,1283.0,0.0,3329.0,193.0,5176.0,33.000000,False,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,151.0,70.0,303.0,565.0,2.0,1091.0,16.000000,True,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,0.0,0.0,0.0,0.0,0.0,0.0,34.000000,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
12966,17.0,847.0,0.0,10.0,144.0,1018.0,42.000000,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
12967,0.0,0.0,0.0,0.0,0.0,0.0,28.771969,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
12968,0.0,2680.0,0.0,0.0,523.0,3203.0,28.771969,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [20]:
train = train_test[train_test['Transported'].notnull()].copy()

In [21]:
train.Transported =train.Transported.astype('int')

In [22]:
test = train_test[train_test['Transported'].isnull()].drop("Transported",axis=1)

In [23]:
X = train.drop('Transported',axis=1)
y = train.Transported

In [24]:
drop_list=['ShoppingMall','Age','CryoSleep_True','HomePlanet_Earth','HomePlanet_Europa',
'VIP_True','HomePlanet_Mars','Destination_PSO J318.5-22','VIP_False',
'Destination_55 Cancri e','FoodCourt','Destination_TRAPPIST-1e']

In [25]:
X=X.drop(drop_list,axis=1)
test=test.drop(drop_list,axis=1)

In [26]:
params_xgb_best= {'lambda': 3.0610042624477543, 
             'alpha': 4.581902571574289, 
             'colsample_bytree': 0.9241969052729379, 
             'subsample': 0.9527591724824661, 
             'learning_rate': 0.06672065863100594, 
             'n_estimators': 730,
             'max_depth': 5, 
             'min_child_weight': 1, 
             'num_parallel_tree': 1}

In [27]:
X,y = shuffle(X,y, random_state=42)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [28]:
def get_score(model,X,y):
    n = cross_val_score(model,X,y,scoring ='accuracy',cv=20)
    return n

In [29]:
print(get_score(xgb.XGBClassifier(**params_xgb_best),X,y).mean())

0.8091683881561522


Initial score is 0.80825. Kaggle Score is 0.81552. Reran with core OLD code back in and got score 0.80917. Kaggle score is 0.81575. Then took out guide section and score is still 0.80917. Kaggle score is 0.81529. 

Using pd.concat for train and test vs. append didn't change the score. Changing Room to Group didn't change the score. No change from breaking out Cabin into Deck, Number, Side, vs Cabin 1, Cabin 2, Cabin 3. Eliminating Name at 'Unknown Unknown' for NaNs didn't change the score. 

Replacing using SimpleImputer with Mode for all NaNs improved the score to 0.80859. Kaggle score is 0.81435. 

Eliminating the guide sections changed the score to 0.80836. Kaggle score is 0.81295.

Using factorization instead of one hot encoder for the categorical data improved the score to 0.81078. Kaggle score is 0.81084. 

Eliminating the dropped columns reduces the score to 0.80652. Using a validation split at 25% reduced the score to 0.79393. Reducing the split to 5% reduced the score to 0.77241. 

Removing the all the expense columns going to zero if CryoSleep is true didn't change the score. 

Removing the Expense column going to zero if CryoSleep is true improved the score to 0.81043. I ran this the second time and it got a score of 0.81216. The Kaggle score was 0.81108. 

Adding back one hot encoding, keeping mode for NaNs and keeping no zero expenses for CryoSleep has a model score of 0.80801. The Kaggle score is 0.81201.

Eliminating VIP improved the score to 0.81066. 

In [30]:
perm = PermutationImportance(xgb.XGBClassifier(**params_xgb_best), random_state=1,n_iter =10,cv=5).fit(X, y)
eli5.show_weights(perm, feature_names = X.columns.tolist(),top=50)

Weight,Feature
0.0757 ± 0.0425,Expenses
0.0728 ± 0.0178,Spa
0.0708 ± 0.0215,RoomService
0.0651 ± 0.0115,VRDeck
0.0362 ± 0.0199,CryoSleep_False
0.0105 ± 0.0060,Deck_C
0.0063 ± 0.0049,Deck_E
0.0058 ± 0.0081,Side_P
0.0045 ± 0.0053,Deck_B
0.0035 ± 0.0069,Deck_F


Looking at all features, the ones over 0.05 are: CryoSleep_True, CryoSleep_False, Expenses, VRDeck, Spa, RoomService, FoodCourt. I wonder if all these sub-expenses are too correlated with Expenses. What if we only included CryoSleep_True, CryoSleep_False, and Expenses?

## **##### 6. Submission**

In [32]:
pred_xgb_best = (xgb.XGBClassifier(**params_xgb_best).fit(X,y)).predict(test)
sample['Transported'] = pred_xgb_best

#This converts the numbers to True/False values
sample['Transported']=sample['Transported']>0.5
sample.to_csv('submit_xgb_best_data.csv', index=False)

Public score came up to 0.81575 which is the 98th percentile. When I made the changes to improve the model score to 0.81009, the Kaggle score dropped to 0.81014. Making the changes to improve model score to 0.81043 reduced the Kaggle score to 0.80967. 

I reran the baseline model of 0.80825 and the Kaggle score reduced slightly to 0.81552. Taking out the Guide sections reduced this to the current score of 0.81529.